In [1]:
from scipy import stats
import numpy as np
import plotly.express as px
import ipywidgets as widgets

In [2]:
def simulate_p_values(n_experiments=1000, n_tosses=50, p_true=0.5):
    """Simulate many coin experiments under the null hypothesis and collect p-values"""
    p_values = []
    binomial = stats.binom(n=n_tosses, p=p_true)
    
    for _ in range(n_experiments):
        # Simulate one experiment: n_tosses of a fair coin
        heads = binomial.rvs()
        
        # Calculate TWO-SIDED p-value: probability of result this extreme or more
        # For binomial: min(P(X ≥ heads), P(X ≤ heads)) × 2
        p_value = 1-binomial.cdf(heads)  # P(X ≥ heads)
        p_value_other_side = binomial.cdf(heads)  # P(X ≤ heads)
        two_sided_p = 2 * min(p_value, p_value_other_side)
        p_values.append(two_sided_p)
    
    return np.array(p_values)

In [3]:
# Run simulation
simulate_p_values(n_experiments=3)

array([0.11892045, 0.03283914, 0.47988766])

In [4]:
def simulate_p_values(n_experiments=1000, n_tosses=50, p_true=0.5):
    """Simulate many coin experiments under the null hypothesis and collect p-values"""
    test_coin = stats.binom(n=n_tosses, p=p_true)    
    fair_coin = stats.binom(n=n_tosses, p=0.5)    

    # Simulate one experiment: n_tosses of a fair coin
    heads = test_coin.rvs(n_experiments)
    #heads = np.random.binomial(n_tosses, p_true, n_experiments)
    
    # Calculate TWO-SIDED p-value: probability of result this extreme or more
    # For binomial: min(P(X ≥ heads), P(X ≤ heads)) × 2
    p_value = 1-fair_coin.cdf(heads)  # P(X ≥ heads)
    p_value_other_side = fair_coin.cdf(heads)  # P(X ≤ heads)
    two_sided_p = 2 * np.min([p_value, p_value_other_side], axis=0)

    return two_sided_p

In [5]:
true_H0_pvalues = simulate_p_values(1000000, 50, 0.5)
significance_level  = 0.25
type_i_error_rate = len(true_H0_pvalues[true_H0_pvalues<significance_level])/len(true_H0_pvalues)

print(f'Type I Error Rate is : {type_i_error_rate:.2f}')

Type I Error Rate is : 0.26


In [6]:
def plot_p_value_distribution(n_experiments=1000, n_tosses=50, alpha=0.05):
    p_values = simulate_p_values(n_experiments, n_tosses)    
    fig = px.histogram(
        x=p_values, 
        nbins=20, 
        title=f"Distribution of P-values under Null Hypothesis (Fair Coin)<br>n_tosses={n_tosses}, α={alpha}",
        labels={'x': 'P-value', 'y': 'Count'})
    # Add significance level line
    fig.add_vline(x=alpha, line_dash="dash", line_color="red", 
                  annotation_text=f"α = {alpha:.2f}", annotation_position="top right")
    # Color the rejection region
    fig.add_vrect(x0=0, x1=alpha, fillcolor="red", opacity=0.2, 
                  annotation_text="Rejection<br>Region", annotation_position="top left")
    # Calculate empirical Type I error rate
    type_i_rate = sum(np.array(p_values) <= alpha) / len(p_values)
    fig.add_annotation(text=f"Empirical Type I Error Rate: {type_i_rate:.3f}",
                      xref="paper", yref="paper", x=0.95, y=0.95, showarrow=False)
    fig.update_traces(xbins=dict(start=0, end=1, size=0.2)) # Fixed bin size   
    return fig

# Make it interactive!
widgets.interact(plot_p_value_distribution, 
                 n_experiments=widgets.IntSlider(min=100, max=200_000, value=1000, step=100),
                 n_tosses=widgets.IntSlider(min=10, max=200, value=50, step=10),
                 alpha=widgets.FloatSlider(min=0.01, max=0.2, value=0.05, step=0.01))

interactive(children=(IntSlider(value=1000, description='n_experiments', max=200000, min=100, step=100), IntSl…

<function __main__.plot_p_value_distribution(n_experiments=1000, n_tosses=50, alpha=0.05)>

In [7]:
def plot_p_value_distribution(n_experiments=1000, n_tosses=50, alpha=0.05, p_true=0.5):
    p_values = simulate_p_values(n_experiments, n_tosses, p_true)    
    fig = px.histogram(
        x=p_values, 
        nbins=20, 
        title=f"Distribution of P-values under p={p_true:.2f} <br>n_tosses={n_tosses}, α={alpha}",
        labels={'x': 'P-value', 'y': 'Count'})
    # Add significance level line
    fig.add_vline(x=alpha, line_dash="dash", line_color="red", 
                  annotation_text=f"α = {alpha:.2f}", annotation_position="top right")
    # Color the rejection region
    fig.add_vrect(x0=0, x1=alpha, fillcolor="red", opacity=0.2, 
                  annotation_text="Rejection<br>Region", annotation_position="top left")
    # Calculate empirical Type II error rate
    type_ii_rate = sum(np.array(p_values) >= alpha) / len(p_values)
    fig.add_annotation(text=f"Empirical Type II Error Rate: {type_ii_rate:.3f}",
                      xref="paper", yref="paper", x=0.95, y=0.95, showarrow=False)
    fig.update_traces(xbins=dict(start=0, end=1, size=0.1)) # Fixed bin size   
    return fig

# Make it interactive!
widgets.interact(plot_p_value_distribution, 
                 n_experiments=widgets.IntSlider(min=100, max=20_000, value=1000, step=100),
                 n_tosses=widgets.IntSlider(min=10, max=500, value=50, step=10),
                 alpha=widgets.FloatSlider(min=0.01, max=0.2, value=0.05, step=0.01),
                 p_true=widgets.FloatSlider(min=0.01, max=1, value=0.5, step=0.01),
                 )

interactive(children=(IntSlider(value=1000, description='n_experiments', max=20000, min=100, step=100), IntSli…

<function __main__.plot_p_value_distribution(n_experiments=1000, n_tosses=50, alpha=0.05, p_true=0.5)>